In [12]:
def get_marketaux_news(company_name: str) -> str:

    import requests
    
    API_KEY = "your_marketaux_api_key"
    url = "https://api.marketaux.com/v1/news/all"
    params = {
        "api_token": "A3k62V7qsceE4p8tM4hTO053p1RxjcVJ8xqdy0zV",
        "symbols": company_name,
        "language": "en",
        "limit": 5
    }
    response = requests.get(url, params=params)
    data = response.json()
    if "data" in data:
        return [article["url"] for article in data["data"]]
    else:
        return []


company_name = "NVDA"

news_urls = get_marketaux_news(company_name)
print(news_urls)

['https://www.benzinga.com/25/03/44577082/nvda-mrna-lcid-domh-tsla-top-5-trending-stocks-today', 'https://economictimes.indiatimes.com/markets/stocks/news/sp-nasdaq-post-big-quarterly-percentage-drops-on-trump-tariff-turmoil/articleshow/119836838.cms', 'https://timesofindia.indiatimes.com/business/international-business/sp-nasdaq-log-worst-quarterly-slumps-in-years-as-trump-tariff-fears-rattle-markets/articleshow/119835888.cms']


In [13]:
def scrape_article(url: str) -> str:
    from newspaper import Article

    try:
        article = Article(url)
        article.download()
        article.parse()
        return {
            "title": article.title,
            "content": article.text,
        }
    except Exception as e:
        print(f"Error scraping article: {e}")
        return {
            "title": "",
            "content": "",
        }

articles = [scrape_article(url) for url in news_urls]

In [14]:
for article in articles:
    print(f"title:{article['title']} len:{len(article['content'])}")

title:Nvidia, Moderna, Lucid, Dominari Holdings, Tesla: Why These 5 Stocks Are On Investors' Radars Today - Dominari Holdings (NASDAQ:DOMH) len:2633
title:S&P, Nasdaq post big quarterly percentage drops on Trump tariff turmoil len:3899
title:S&P, Nasdaq log worst quarterly slumps in years as Trump tariff fears rattle markets len:3586


In [15]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import BaseLLM
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

def get_summary_from_llm(llm: BaseLLM, config: dict, article_title: str, article_content: str) -> str:
    prompt = f"""you are a professional journalist who has been asked to analyze the following article and provide a summary, sentiment analysis, and key points.
        Analyze the following article and provide a summary, sentiment analysis, and key points:

        Title: {article_title}
        Content: {article_content}

        Please provide your analysis in the following format:
        
        Sentiment: The overall sentiment score (positive:10, negative:1, or neutral:5)
        Summary: A concise summary of the article (2-3 sentences)
        
    """

    return llm.invoke(input=prompt, config=config).content


In [16]:

def get_human_message(article_title: str, article_content: str) -> str:
    prompt = f"""you are a professional journalist who has been asked to analyze the following article and provide a summary, sentiment analysis, and key points.
        Analyze the following article and provide a summary, sentiment analysis, and key points:

        Title: {article_title}
        Content: {article_content}

        Please provide your analysis in the following format:
        
        Sentiment: The overall sentiment score (positive:10, negative:1, or neutral:5)
        Summary: A concise summary of the article (2-3 sentences)
        Key Points: The key points of the article (3-5 bullet points)
        
    """
    return prompt

In [17]:
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

config = {"configurable": {"thread_id": "1234"}}
llm = init_chat_model("gpt-4o-mini", model_provider="openai", store=True)

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

i = 0
for article in articles:
    i = i+1
    print(f"\n--{i}--\n")

    if article['content']:
        print(f"Title: {article['title']}")
        
        input_messages = [HumanMessage(get_human_message(article['title'], article['content']))]
        output = app.invoke({"messages": input_messages}, config)
        output["messages"][-1].pretty_print()
        
    else:
        print(f"Skipping article '{article['title']}' due to empty content.")


/var/folders/v_/495_mfbd4rv76fxlb3w_3hs80000gn/T/ipykernel_4310/2875842115.py:7: UserWarning: WARNING! store is not default parameter.
                store was transferred to model_kwargs.
                Please confirm that store is what you intended.
  llm = init_chat_model("gpt-4o-mini", model_provider="openai", store=True)



--1--

Title: Nvidia, Moderna, Lucid, Dominari Holdings, Tesla: Why These 5 Stocks Are On Investors' Radars Today - Dominari Holdings (NASDAQ:DOMH)
================================== Ai Message ==================================

**Sentiment:** 4 (negative)

**Summary:** The stock market experienced volatility, with the S&P 500 struggling and facing its worst month since September 2022. Key stocks including Nvidia, Moderna, Lucid, Dominari Holdings, and Tesla are highlighted, each facing distinct challenges or gains amid a generally negative market sentiment.

**Key Points:**
- The S&P 500 ended the day up 0.5% but is on track for a 7% decline in March, marking the worst month since September 2022.
- Nvidia's stock fell 1.18% despite plans for significant U.S. supply chain investments.
- Moderna saw an 8.90% drop, reaching a new 52-week low following a major change at the FDA.
- Lucid's stock rose 4.31% amid positive news regarding increased orders from ex-Tesla customers.
- Dominari 

In [6]:
prompt = "What's your overall sentiment about these articles? (positive:10, negative:1, or neutral:5). Only Score/10 is required."

input_messages = [HumanMessage(prompt)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


================================== Ai Message ==================================

3/10
